In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transforms3d as tf3d

# Generating body acc, grad ned acc, grad body acc

In [2]:

def gen_grad_acc(df):
    
    df3=df
    
    acc_ned_grad=np.zeros((len(df3),3))

    acc_ned_grad[:,0]=np.diff(df3['speed[0]'],append=0)/np.diff(df3["t"],append=4e-3)

    acc_ned_grad[:,0]=np.array([i  if abs(i)<30 else 0 for i in acc_ned_grad[:,0]])

    acc_ned_grad[:,1]=np.diff(df3['speed[1]'],append=0)/np.diff(df3["t"],append=4e-3)

    acc_ned_grad[:,1]=np.array([i  if abs(i)<30 else 0 for i in acc_ned_grad[:,1]])


    acc_ned_grad[:,2]=np.diff(df3['speed[2]'],append=0)/np.diff(df3["t"],append=4e-3)

    acc_ned_grad[:,2]=np.array([i  if abs(i)<30 else 0 for i in acc_ned_grad[:,2]])

    acc_body_grad=np.zeros((len(df3),3))

    for i in df3.index:
        q0,q1,q2,q3=df3["q[0]"][i],df3["q[1]"][i],df3["q[2]"][i],df3["q[3]"][i]
    #     print(i,q0,q1,q2,q3)
        R=tf3d.quaternions.quat2mat(np.array([q0,q1,q2,q3]))
        acc_body_grad[i]=R.T@(acc_ned_grad[i].reshape((3,1))).flatten()
        

    R_array=np.array([tf3d.quaternions.quat2mat([i,j,k,l]) for i,j,k,l in zip(df['q[0]'],df['q[1]'],df['q[2]'],df['q[3]'])])
    v_ned_array=np.array([df['speed[%i]'%(i)] for i in range(3)]).T
    v_body_array=np.array([(i.T@(j.T)).T for i,j in zip(R_array,v_ned_array)])
    gamma_array=np.array([(i.T@(np.array([0,0,9.81]).T)).T for i in R_array])



    df3['acc_ned_grad[0]'],df3['acc_ned_grad[1]'],df3['acc_ned_grad[2]']=acc_ned_grad.T
    df3['acc_body_grad[0]'],df3['acc_body_grad[1]'],df3['acc_body_grad[2]']=acc_body_grad.T
    
    for i in range(3):
        df3['speed_body[%i]'%(i)]=v_body_array[:,i]
        df3['gamma[%i]'%(i)]=gamma_array[:,i]
    return df3



In [3]:
log=pd.read_csv('./logs/target/log_real.csv',low_memory=False).astype(float)

In [4]:
log['t']=pd.to_numeric(log['t'])

In [5]:
log['t'][65517:65521]

65517    276.732792
65518    276.736842
Name: t, dtype: float64

In [6]:
log_processed=gen_grad_acc(log)

In [7]:
log_processed.to_csv('./logs/target/log_real_processed.csv')